In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sympy
from sympy import Derivative, Function, Symbol, symbols, Eq, pi, cos, sin, exp, log, oo
from sympy import Function, dsolve, Derivative, checkodesol
from sympy import fps, Rational
from sympy import pprint, Matrix, eye, zeros
from sympy import Inverse
from utils import *
z = Symbol('z')

Restate the problem that we want to solve: 

$$z^k \frac{d\psi}{dz} = A(z)\psi$$, where $\psi$ is a vector-valued function and $A(z)$ as a given matrix-valued function is holomorphic.

In the general case where we can't find it directly, we use Formal Gauge transform to simplify the system, and solve it in diagonal version. Then, $\hat{\Phi}(u, z) = \hat{\Psi}(t(u, z)) \hat{\Psi}^{-1}(s(u,z))$ is the formal universal solution on Sto_k, while since $\Phi$ is unique, then this means that $\hat{\Phi}$ is the power series of $\Phi$, so it must be convergent since $\Phi$ is entire.

To do the Gauge transformation, the core idea is to transfer it to the case where $z^k \frac{d\phi}{dz} = (D_0+zD_1+...z^{k-1}D_{k-1})\phi$, where $D_i$ are diagonal, which can be easily solve by entries on both sides.

In order to solve this problem, we first need to make a change of variables $\psi' = F \psi$, whereas $F = 1+zF_1 + z^2F_2 + ...$









In this step, we let $F = ......(I+z^2H_2)(I+zH_1)$

where $H_p = \begin{cases} {ad}^{-1}_{A_0}(A_p^{OD}) \text{   if   } k > 1 \\ ({ad}_{A_0} - p)^{-1}(A_p^{OD}) \text{    if   } k = 1  \end{cases}$

After this transform, the ODE becomes $z^k \frac{d\psi'}{dz} = (D_0 + D_1 z + ...) \psi'$

After doing this, we make the second Gauge Transform to transform it to the finite case, 

which is the transform $K = \exp(-\int{D_k+D_{k+1}z+...})$


Finally, the simplified system is $z^k \frac{d\psi''}{dz} = (D_0 + D_1 z + ...+ D_{k-1} z^{k-1}) \psi''$

and the overall Gauge transform is $(KF)$[...], and the inverse is $(KF)^{-1}$[...], so $\psi = (KF)^{-1}\psi''$

After we get the fundamental solution $\Psi$, we can construct $\Phi$ on the Sto_k in a similar way.

However, in order for the above approach to work, we have to impose the condition that the ODE is generic.

This means that the leading term $A(0)$ is diagonalizable, with eigenvalues $(\alpha_1, ..., \alpha_n)$ satisfying that 

1. if $k \geq 2$, then they are distinct 

2. if $k = 1$, they no pair of eigenvalues differs by a positive integer.

This is known as the non-resonance assumption.

Example 1: 

$z \psi' = \begin{pmatrix} \frac{1}{2} & z \\ 0 & 1 \end{pmatrix} \psi$ 

We are working on Sto_1

We trancate the Gauge Transform series up to some given order 

Since this matrix is upper triangular, it can be solved directly:

Firstly solve $\psi_2$, which is $Az$, 

then substitute this into the equation of $\psi_1$ and get $z\psi'_1 = \frac{1}{2}\psi_1 + Az^2$, then we get

$\psi_1 = \frac{2A}{3}z^2 + B \sqrt{z}$, where $A$, $B$ are constants

so $\psi = \begin{pmatrix} \frac{2A}{3}z^2 + B \sqrt{z} \\ Az \end{pmatrix} = A\begin{pmatrix} \frac{2}{3}z^2 \\ z \end{pmatrix} + B\begin{pmatrix} \sqrt{z} \\ 0 \end{pmatrix}$

In our method, we get $F = (I + \begin{pmatrix} 0 & -\frac{2}{3} \\ 0 & 0 \end{pmatrix}z)$

In [2]:
A = Matrix([[Rational(1,2), z], [0, 1]])
n = 2
k = 1
order = 5

In [3]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [4]:
G = Gauge(k, Total, A)
G

Matrix([
[1/2, 0],
[  0, 1]])

In [5]:
Total

Matrix([
[1, -2*z/3],
[0,      1]])

In [6]:
Total_inverse = Total.inverse()

In [7]:
Total_inverse

Matrix([
[1, 2*z/3],
[0,     1]])

In [8]:
Psi_prime, Phi_prime = solve_phi(k, G)

In [9]:
Psi_prime

Matrix([
[sqrt(z), 0],
[      0, z]])

In [10]:
Psi = Total_inverse * Psi_prime
Psi = Psi.simplify()

In [11]:
Psi

Matrix([
[sqrt(z), 2*z**2/3],
[      0,        z]])

This value of $\Psi$ matches exactly with our manually found solution stated in the beginning!!

In [12]:
Final_Phi = get_phi_from_psi(Psi, k)

In [13]:
sympy.simplify(Final_Phi)

Matrix([
[sqrt(z*exp(u))/sqrt(z), -2*sqrt(z)*sqrt(z*exp(u))/3 + 2*z*exp(2*u)/3],
[                     0,                                       exp(u)]])

This is our final universal solution in the Stokes groupoid

Example2:

Note that in the generic case, $A(0)$ is required to be diagonaliable, yet may not be diagonal, because we could add a Gauge Transform to make it diagonal.

For example: $z^2 \psi' = \begin{pmatrix} 1 & z+1 \\ 1 & z \end{pmatrix} \psi$



In [14]:
A = Matrix([[1, z+1], [1, z]])
n = 2
k = 2
order = 3

In [15]:
A0 = A.subs(z, 0)
W, L = A0.diagonalize()
assert(L == (Inverse(W)*A0*W).simplify())
assert(A0 == (W*L*Inverse(W)).simplify())
# Since we know that the Gauge transform of A
# is FAF^{-1} + z^k F' F^{-1} 
# then we could let F = W^{-1} since F' = 0 since W is constant, 
# to transform A0 to L 

In [16]:
newA = (Inverse(W)*A*W).simplify()

In [17]:
newA

Matrix([
[-sqrt(5)*z/10 + z/2 - sqrt(5)/2 + 1/2,                  -sqrt(5)*z/10 + z/2],
[                   z*(sqrt(5) + 5)/10, sqrt(5)*z/10 + z/2 + 1/2 + sqrt(5)/2]])

In [18]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(newA, k, order)
## Takes 2 minutes

In [19]:
F_trunc
K_trunc
Total

Matrix([
[-sqrt(5)*z**3*(sqrt(5)/50 + 7/50)*(2*z**2/25 + sqrt(5)*z*(sqrt(5)/10 + 1/2)/5)/5 + 2*sqrt(5)*z**3*(sqrt(5)/10 + 1/2)/125 + 1, -sqrt(5)*z**3*(sqrt(5)/50 + 7/50)*(-2*sqrt(5)*z**3*(1/2 - sqrt(5)/10)/125 + 1)/5 + 2*z**2/25 - sqrt(5)*z*(1/2 - sqrt(5)/10)/5],
[ sqrt(5)*z**3*(7/50 - sqrt(5)/50)*(2*sqrt(5)*z**3*(sqrt(5)/10 + 1/2)/125 + 1)/5 + 2*z**2/25 + sqrt(5)*z*(sqrt(5)/10 + 1/2)/5,   sqrt(5)*z**3*(7/50 - sqrt(5)/50)*(2*z**2/25 - sqrt(5)*z*(1/2 - sqrt(5)/10)/5)/5 - 2*sqrt(5)*z**3*(1/2 - sqrt(5)/10)/125 + 1]])

Matrix([
[exp(-z*(sqrt(5)*z + 5*z - 10*sqrt(5))/250),                                           0],
[                                         0, exp(-z*(-sqrt(5)*z + 5*z + 10*sqrt(5))/250)]])

Matrix([
[       (-sqrt(5)*z**4*(sqrt(5) + 7)*(4*z + sqrt(5)*(sqrt(5) + 5)) + 20*sqrt(5)*z**3*(sqrt(5) + 5) + 12500)*exp(-z*(sqrt(5)*z + 5*z - 10*sqrt(5))/250)/12500, z*(sqrt(5)*z**2*(sqrt(5) + 7)*(sqrt(5)*z**3*(5 - sqrt(5)) - 625) + 12500*z - 3125*sqrt(5)*(5 - sqrt(5)))*exp(-z*(sqrt(5)*z + 5*z - 10*sqrt(5))/250)/156250],
[z*(sqrt(5)*z**2*(7 - sqrt(5))*(sqrt(5)*z**3*(sqrt(5) + 5) + 625) + 12500*z + 3125*sqrt(5)*(sqrt(5) + 5))*exp(-z*(-sqrt(5)*z + 5*z + 10*sqrt(5))/250)/156250,      (sqrt(5)*z**4*(7 - sqrt(5))*(4*z - sqrt(5)*(5 - sqrt(5))) + 20*sqrt(5)*z**3*(-5 + sqrt(5)) + 12500)*exp(-z*(-sqrt(5)*z + 5*z + 10*sqrt(5))/250)/12500]])

In [20]:
transformed_newA = Gauge(k, Total, newA)
transformed_newA

Matrix([
[(-704*sqrt(5)*z**20 - 1056*z**20 - 1760*sqrt(5)*z**19 - 2640*z**19 + 63800*z**18 + 90640*sqrt(5)*z**18 + 170500*sqrt(5)*z**17 + 544500*z**17 + 990000*z**16 + 1837000*sqrt(5)*z**16 + 4997500*sqrt(5)*z**15 + 18647500*z**15 - 11231250*sqrt(5)*z**14 - 175000*z**14 - 96515625*z**13 - 13796875*sqrt(5)*z**13 - 176515625*sqrt(5)*z**12 - 204140625*z**12 - 2085546875*z**11 - 455078125*sqrt(5)*z**11 + 2111328125*z**10 + 1483984375*sqrt(5)*z**10 + 33203125*sqrt(5)*z**9 + 3654296875*z**9 - 3037109375*sqrt(5)*z**8 - 2587890625*z**8 + 6347656250*sqrt(5)*z**7 + 92089843750*z**7 - 49804687500*sqrt(5)*z**6 + 16113281250*z**6 - 145263671875*z**5 - 20751953125*sqrt(5)*z**5 + 6103515625*z**4 + 152587890625*sqrt(5)*z**4 - 30517578125*sqrt(5)*z**3 + 30517578125*z**3 - 152587890625*sqrt(5)*z**2 + 152587890625*z**2 - 762939453125*sqrt(5)*z + 3814697265625*z - 3814697265625*sqrt(5) + 3814697265625)/(50*(220*z**17 + 308*sqrt(5)*z**17 + 660*sqrt(5)*z**16 + 2200*z**16 + 4400*z**15 + 6600*sqrt(5)*z**15 + 

In [21]:
# These are its first 4 coefficients (z**0 to z**3) in its Taylor expansion.
transformed_newA.subs(z, 0)
transformed_newA.diff(z).subs(z, 0)
transformed_newA.diff(z,z).subs(z, 0)
transformed_newA.diff(z,z,z).subs(z, 0)
# Indeed, we can see that it is indeed diagonal up to order z**3, where 3 is the order we set.
# Furthermore, due to the functionality of K, the terms after k until order are all zero
# Hence, as order goes to infinite, the transformed ode is just a polynomial of order k-1. 

Matrix([
[1/2 - sqrt(5)/2,               0],
[              0, 1/2 + sqrt(5)/2]])

Matrix([
[1/2 - sqrt(5)/10,                0],
[               0, sqrt(5)/10 + 1/2]])

Matrix([
[0, 0],
[0, 0]])

Matrix([
[-12/125 + 3*(3814697265625 - 3814697265625*sqrt(5))*(-244140625*sqrt(5) - 244140625)/116415321826934814453125,                                                                                                                            0],
[                                                                                                            0, 3*(3814697265625 + 3814697265625*sqrt(5))*(-244140625*sqrt(5) - 244140625)/116415321826934814453125 + 6*sqrt(5)/125 + 18/125]])

In [ ]:
Total_inverse = Total.inverse()
Total_inverse

Matrix([[(-3437500*z**10*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) - 8593750*z**9*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) + 44921875*z**8*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) + 19531250*z**7*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) - 39062500*z**6*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) - 488281250*z**5*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) - 4882812500*z**4*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) + 2441406250*z**3*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25) + 152587890625*exp(-z**2/50 - sqrt(5)*z**2/250 + sqrt(5)*z/25))/(220*z**17*exp(-z**2/25 - sqrt(5)*z**2/125 + 2*sqrt(5)*z/25) + 308*sqrt(5)*z**17*exp(-z**2/25 - sqrt(5)*z**2/125 + 2*sqrt(5)*z/25) + 660*sqrt(5)*z**16*exp(-z**2/25 - sqrt(5)*z**2/125 + 2*sqrt(5)*z/25) + 2200*z**16*exp(-z**2/25 - sqrt(5)*z**2/125 + 2*sqrt(5)*z/25) + 4400*z**15*exp(-z**2/25 - sqrt(5)*z**2/125 + 2*sqrt(5)*z/25) + 6600*sqrt(5)*z**15*exp(-z**2/25 - sqrt(5)*z**2/125 + 2*sqrt(5)*z/25) + 16500*sqrt(

In [23]:

Psi_prime, Phi_prime = solve_phi(k, transformed_newA)
Psi = W * Total_inverse * Psi_prime
# We transform newA back by W
Psi = Psi.simplify()

In [24]:
Psi_prime
Psi

Matrix([
[z**(1/2 - sqrt(5)/10)*exp((-1 + sqrt(5))/(2*z)),                                               0],
[                                              0, z**(sqrt(5)/10 + 1/2)*exp(-(1 + sqrt(5))/(2*z))]])

Matrix([
[15625*z**(1/2 - sqrt(5)/10)*(44*sqrt(5)*z**11 + 132*z**11 + 220*z**10 + 440*sqrt(5)*z**10 - 1050*z**9 + 370*sqrt(5)*z**9 - 125*sqrt(5)*z**8 + 5625*z**8 + 8625*sqrt(5)*z**7 + 31875*z**7 + 15000*sqrt(5)*z**6 + 35000*z**6 + 71875*z**5 + 78125*sqrt(5)*z**5 - 437500*z**4 + 250000*sqrt(5)*z**4 - 1015625*z**3 - 234375*sqrt(5)*z**3 - 781250*sqrt(5)*z**2 - 781250*z**2 - 5859375*z - 1953125*sqrt(5)*z - 9765625*sqrt(5) + 9765625)*exp(sqrt(5)*z**2/250 + z**2/50 - sqrt(5)*z/25 - 1/(2*z) + sqrt(5)/(2*z))/(2*(220*z**17 + 308*sqrt(5)*z**17 + 660*sqrt(5)*z**16 + 2200*z**16 + 4400*z**15 + 6600*sqrt(5)*z**15 + 16500*sqrt(5)*z**14 + 55000*z**14 - 75625*sqrt(5)*z**13 - 61875*z**13 - 481250*z**12 - 103125*sqrt(5)*z**12 - 943750*sqrt(5)*z**11 - 625000*z**11 - 11406250*z**10 - 2390625*sqrt(5)*z**10 + 5546875*z**9 + 6171875*sqrt(5)*z**9 + 4687500*sqrt(5)*z**8 + 37109375*z**8 - 21484375*sqrt(5)*z**7 - 17578125*z**7 - 29296875*sqrt(5)*z**6 + 400390625*z**6 - 292968750*sqrt(5)*z**5 - 195312500*z**5 - 34

In [28]:
# Final_Phi = get_phi_from_psi(Psi, k)

Example3: 

$z^2 \psi' = \begin{pmatrix} \cos(z)+1 & \sin(z) \\ -\sin(z) & \cos(z) \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [25]:
A = Matrix([[cos(z)+1, sin(z)], [-sin(z), cos(z)]])
n = 2
k = 2
order = 3
# This means that we only approaching using the first 3 terms

In [26]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)
# Takes 8 minutes

In [28]:
F_trunc
K_trunc
Total

Matrix([
[z**3*(-17*z**2/6 + 17*z/6) + z**3 + 1,   z**3*(17/6 - 17*z**3/6) + z**2 + z],
[   z**3*(17*z**3/6 + 17/6) - z**2 + z, z**3*(17*z**2/6 + 17*z/6) - z**3 + 1]])

Matrix([
[exp(z*(z + 3)/2),                0],
[               0, exp(z*(z - 1)/2)]])

Matrix([
[   (z**4*(17 - 17*z)/6 + z**3 + 1)*exp(z*(z + 3)/2), z*(17*z**2*(1 - z**3) + 6*z + 6)*exp(z*(z + 3)/2)/6],
[z*(17*z**2*(z**3 + 1) - 6*z + 6)*exp(z*(z - 1)/2)/6,    (z**4*(17*z + 17)/6 - z**3 + 1)*exp(z*(z - 1)/2)]])

In [29]:
transformed_A = Gauge(k, Total, A)

In [31]:
# These are its first 4 coefficients (z**0 to z**4) in its Taylor expansion.
transformed_A.subs(z, 0)
transformed_A.diff(z).subs(z, 0)
transformed_A.diff(z,z).subs(z, 0)
transformed_A.diff(z,z,z).subs(z, 0)
transformed_A.diff(z,z,z,z).subs(z, 0)
# Indeed, we can see that it is indeed diagonal up to order z**3, where 3 is the order we set.
# Furthermore, due to the functionality of K, the terms after k until order are all zero
# Hence, as order goes to infinite, the transformed ode is just a polynomial of order k-1. 

Matrix([
[2, 0],
[0, 1]])

Matrix([
[0, 0],
[0, 0]])

Matrix([
[0, 0],
[0, 0]])

Matrix([
[0, 0],
[0, 0]])

Matrix([
[  9, 276],
[276,  -7]])

In [32]:
Total_inverse = Total.inverse()

In [33]:
Total_inverse

Matrix([
[(102*z**5*exp(z**2/2 + 3*z/2) + 102*z**4*exp(z**2/2 + 3*z/2) - 36*z**3*exp(z**2/2 + 3*z/2) + 36*exp(z**2/2 + 3*z/2))/(289*z**12*exp(z**2 + 3*z) - 289*z**10*exp(z**2 + 3*z) + 289*z**8*exp(z**2 + 3*z) - 325*z**6*exp(z**2 + 3*z) + 36*z**4*exp(z**2 + 3*z) - 36*z**2*exp(z**2 + 3*z) + 36*exp(z**2 + 3*z)), (102*z**6 - 102*z**3 - 36*z**2 - 36*z)/(289*z**12*exp(z**2/2 - z/2) - 289*z**10*exp(z**2/2 - z/2) + 289*z**8*exp(z**2/2 - z/2) - 325*z**6*exp(z**2/2 - z/2) + 36*z**4*exp(z**2/2 - z/2) - 36*z**2*exp(z**2/2 - z/2) + 36*exp(z**2/2 - z/2))],
[                                                 (-102*z**6 - 102*z**3 + 36*z**2 - 36*z)/(289*z**12*exp(z**2/2 + 3*z/2) - 289*z**10*exp(z**2/2 + 3*z/2) + 289*z**8*exp(z**2/2 + 3*z/2) - 325*z**6*exp(z**2/2 + 3*z/2) + 36*z**4*exp(z**2/2 + 3*z/2) - 36*z**2*exp(z**2/2 + 3*z/2) + 36*exp(z**2/2 + 3*z/2)),  (-102*z**5 + 102*z**4 + 36*z**3 + 36)/(289*z**12*exp(z**2/2 - z/2) - 289*z**10*exp(z**2/2 - z/2) + 289*z**8*exp(z**2/2 - z/2) - 325*z**6*exp(z**2/2 

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = D_0 + D_1 z + ... + D_{k-1}z^{k-1}$

In [34]:
Psi_prime, Phi_prime = solve_phi(k, transformed_A)

In [35]:
Psi_prime

Matrix([
[exp(-2/z),         0],
[        0, exp(-1/z)]])

In [36]:
Phi_prime

Matrix([
[exp(2*(1 - exp(-u*z))/z),                      0],
[                       0, exp((1 - exp(-u*z))/z)]])

The final solution is $T^{-1}\Psi$ (we use $T$ to denote the initial change of coordinate that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

So the final solution $\Psi$ is 

In [37]:
Psi = Total_inverse * Psi_prime

Hence, the original universal solution in the Stokes groupoid of order $k$ is 

In [38]:
Phi = get_phi_from_psi(Psi, k)

In [39]:
Psi
Phi

Matrix([
[(102*z**5*exp(z**2/2 + 3*z/2) + 102*z**4*exp(z**2/2 + 3*z/2) - 36*z**3*exp(z**2/2 + 3*z/2) + 36*exp(z**2/2 + 3*z/2))*exp(-2/z)/(289*z**12*exp(z**2 + 3*z) - 289*z**10*exp(z**2 + 3*z) + 289*z**8*exp(z**2 + 3*z) - 325*z**6*exp(z**2 + 3*z) + 36*z**4*exp(z**2 + 3*z) - 36*z**2*exp(z**2 + 3*z) + 36*exp(z**2 + 3*z)), (102*z**6 - 102*z**3 - 36*z**2 - 36*z)*exp(-1/z)/(289*z**12*exp(z**2/2 - z/2) - 289*z**10*exp(z**2/2 - z/2) + 289*z**8*exp(z**2/2 - z/2) - 325*z**6*exp(z**2/2 - z/2) + 36*z**4*exp(z**2/2 - z/2) - 36*z**2*exp(z**2/2 - z/2) + 36*exp(z**2/2 - z/2))],
[                                                 (-102*z**6 - 102*z**3 + 36*z**2 - 36*z)*exp(-2/z)/(289*z**12*exp(z**2/2 + 3*z/2) - 289*z**10*exp(z**2/2 + 3*z/2) + 289*z**8*exp(z**2/2 + 3*z/2) - 325*z**6*exp(z**2/2 + 3*z/2) + 36*z**4*exp(z**2/2 + 3*z/2) - 36*z**2*exp(z**2/2 + 3*z/2) + 36*exp(z**2/2 + 3*z/2)),  (-102*z**5 + 102*z**4 + 36*z**3 + 36)*exp(-1/z)/(289*z**12*exp(z**2/2 - z/2) - 289*z**10*exp(z**2/2 - z/2) + 289*z**8*

Matrix([
[(-z**2*(17*z**5 + 17*z**2 - 6*z + 6)*(-17*z**5*exp(5*u*z) + 17*z**2*exp(2*u*z) + 6*z*exp(u*z) + 6)*(289*z**12*exp(z*(12*u + z*exp(2*u*z) + 3*exp(u*z))) - 289*z**10*exp(z*(10*u + z*exp(2*u*z) + 3*exp(u*z))) + 289*z**8*exp(z*(8*u + z*exp(2*u*z) + 3*exp(u*z))) - 325*z**6*exp(z*(6*u + z*exp(2*u*z) + 3*exp(u*z))) + 36*z**4*exp(z*(4*u + z*exp(2*u*z) + 3*exp(u*z))) - 36*z**2*exp(z*(2*u + z*exp(2*u*z) + 3*exp(u*z))) + 36*exp(z*(z*exp(u*z) + 3)*exp(u*z)))*exp(u*z + (z**3 - z**2 + 6)/(2*z) + (3*z**3*exp(u*z) + 9*z**2*exp(u*z) + 6*exp(u*z) + 4)*exp(-u*z)/(2*z)) + (-17*z**5 + 17*z**4 + 6*z**3 + 6)*(17*z**5*exp(z*(10*u + z*exp(2*u*z) + 3*exp(u*z))/2) + 17*z**4*exp(z*(8*u + z*exp(2*u*z) + 3*exp(u*z))/2) - 6*z**3*exp(z*(6*u + z*exp(2*u*z) + 3*exp(u*z))/2) + 6*exp(z*(z*exp(u*z) + 3)*exp(u*z)/2))*(289*z**12*exp(z*(24*u + z*exp(2*u*z) - exp(u*z))/2) - 289*z**10*exp(z*(20*u + z*exp(2*u*z) - exp(u*z))/2) + 289*z**8*exp(z*(16*u + z*exp(2*u*z) - exp(u*z))/2) - 325*z**6*exp(z*(12*u + z*exp(2*u*z) -

Example4: 

$z^2 \psi' = \begin{pmatrix} z^4+3z^3+1 & z \\ z^2 & z^3+5z^2+7 \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [40]:
A = Matrix([[z**4+3*z**3+1, z], [z**2, z**3+5*z**2+7]])
n = 2
k = 2
order = 3

In [41]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [43]:
F_trunc
K_trunc
Total

Matrix([
[  7*z**5/324 + 1, z**3*(7/54 - 7*z**3/1944) + z**2/36 - z/6],
[z**3/18 + z**2/6,     z**3*(z**2/648 - z/108) - z**3/36 + 1]])

Matrix([
[exp(-17*z**2/12),                     0],
[               0, exp(-z*(7*z + 60)/12)]])

Matrix([
[  (7*z**5 + 324)*exp(-17*z**2/12)/324, z*(7*z**2*(36 - z**3) + 54*z - 324)*exp(-17*z**2/12)/1944],
[z**2*(z + 3)*exp(-z*(7*z + 60)/12)/18,  (z**4*(z - 6) - 18*z**3 + 648)*exp(-z*(7*z + 60)/12)/648]])

In [44]:
transformed_A = Gauge(k, Total, A)

In [45]:
# These are its first 4 coefficients (z**0 to z**4) in its Taylor expansion.
transformed_A.subs(z, 0)
transformed_A.diff(z).subs(z, 0)
transformed_A.diff(z,z).subs(z, 0)
transformed_A.diff(z,z,z).subs(z, 0)
transformed_A.diff(z,z,z,z).subs(z, 0)
# Indeed, we can see that it is indeed diagonal up to order z**3, where 3 is the order we set.
# Furthermore, due to the functionality of K, the terms after k until order are all zero
# Hence, as order goes to infinite, the transformed ode is just a polynomial of order k-1. 

Matrix([
[1, 0],
[0, 7]])

Matrix([
[0, 0],
[0, 0]])

Matrix([
[0, 0],
[0, 0]])

Matrix([
[0, 0],
[0, 0]])

Matrix([
[74/3,   20],
[ -16, -2/3]])

In [46]:
Total_inverse = Total.inverse()

In [47]:
Total_inverse

Matrix([
[(324*z**5 - 1944*z**4 - 5832*z**3 + 209952)/(7*z**10*exp(-17*z**2/12) - 1512*z**6*exp(-17*z**2/12) - 972*z**4*exp(-17*z**2/12) + 209952*exp(-17*z**2/12)), (756*z**6 - 27216*z**3 - 5832*z**2 + 34992*z)/(7*z**10*exp(-7*z**2/12 - 5*z) - 1512*z**6*exp(-7*z**2/12 - 5*z) - 972*z**4*exp(-7*z**2/12 - 5*z) + 209952*exp(-7*z**2/12 - 5*z))],
[                 (-11664*z**3 - 34992*z**2)/(7*z**10*exp(-17*z**2/12) - 1512*z**6*exp(-17*z**2/12) - 972*z**4*exp(-17*z**2/12) + 209952*exp(-17*z**2/12)),                          (4536*z**5 + 209952)/(7*z**10*exp(-7*z**2/12 - 5*z) - 1512*z**6*exp(-7*z**2/12 - 5*z) - 972*z**4*exp(-7*z**2/12 - 5*z) + 209952*exp(-7*z**2/12 - 5*z))]])

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = D_0 + D_1 z + ... + D_{k-1} z^{k-1}$

In [48]:
Psi_prime, Phi_prime = solve_phi(k, transformed_A)

In [49]:
Psi_prime

Matrix([
[exp(-1/z),         0],
[        0, exp(-7/z)]])

In [50]:
Phi_prime

Matrix([
[exp((1 - exp(-u*z))/z),                        0],
[                     0, exp(7*(1 - exp(-u*z))/z)]])

The final solution is $T^{-1}\Psi$ (we use $T$ to denote the change of coordinate that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

In [52]:
T_inverse_Psi = Total_inverse * Psi_prime
T_inverse_Psi

Matrix([
[(324*z**5 - 1944*z**4 - 5832*z**3 + 209952)*exp(-1/z)/(7*z**10*exp(-17*z**2/12) - 1512*z**6*exp(-17*z**2/12) - 972*z**4*exp(-17*z**2/12) + 209952*exp(-17*z**2/12)), (756*z**6 - 27216*z**3 - 5832*z**2 + 34992*z)*exp(-7/z)/(7*z**10*exp(-7*z**2/12 - 5*z) - 1512*z**6*exp(-7*z**2/12 - 5*z) - 972*z**4*exp(-7*z**2/12 - 5*z) + 209952*exp(-7*z**2/12 - 5*z))],
[                 (-11664*z**3 - 34992*z**2)*exp(-1/z)/(7*z**10*exp(-17*z**2/12) - 1512*z**6*exp(-17*z**2/12) - 972*z**4*exp(-17*z**2/12) + 209952*exp(-17*z**2/12)),                          (4536*z**5 + 209952)*exp(-7/z)/(7*z**10*exp(-7*z**2/12 - 5*z) - 1512*z**6*exp(-7*z**2/12 - 5*z) - 972*z**4*exp(-7*z**2/12 - 5*z) + 209952*exp(-7*z**2/12 - 5*z))]])

In [53]:
Final_Phi = get_phi_from_psi(T_inverse_Psi, k)
Final_Phi
## This will take much long time to run.

Matrix([
[(6*z**3*(z + 3)*(7*z**5*exp(5*u*z) - 252*z**2*exp(2*u*z) - 54*z*exp(u*z) + 324)*exp(u*z + (7*z**3*exp(3*u*z) + 17*z**3*exp(u*z) + 60*z**2*exp(2*u*z) + 96*exp(u*z) + 12)*exp(-u*z)/(12*z)) + (7*z**5 + 324)*(z**5*exp(5*u*z) - 6*z**4*exp(4*u*z) - 18*z**3*exp(3*u*z) + 648)*exp(((17*z**3*exp(3*u*z) + 7*z**3*exp(u*z) + 60*z**2*exp(u*z) + 12*exp(u*z) + 84)*exp(-u*z)/12 + 1)/z))*exp((-(2*z**3*exp(u*z) + 5*z**2*exp(u*z) + 9*exp(u*z) + 8)*exp(-u*z) + 8)/z)/(7*z**10*exp(10*u*z) - 1512*z**6*exp(6*u*z) - 972*z**4*exp(4*u*z) + 209952), z*((z**5 - 6*z**4 - 18*z**3 + 648)*(7*z**5*exp(5*u*z) - 252*z**2*exp(2*u*z) - 54*z*exp(u*z) + 324)*exp(u*z + (7*z**3*exp(3*u*z) + 17*z**3*exp(u*z) + 60*z**2*exp(2*u*z) + 96*exp(u*z) + 12)*exp(-u*z)/(12*z)) - (7*z**5 - 252*z**2 - 54*z + 324)*(z**5*exp(5*u*z) - 6*z**4*exp(4*u*z) - 18*z**3*exp(3*u*z) + 648)*exp(((17*z**3*exp(3*u*z) + 7*z**3*exp(u*z) + 60*z**2*exp(u*z) + 12*exp(u*z) + 84)*exp(-u*z)/12 + 1)/z))*exp((-(2*z**3*exp(u*z) + 5*z**2*exp(u*z) + 9*exp(u*z)

Even though we could Finally get $\Phi$ of the original system, which is clearly smooth, a limitation that we could optimize is its running time, yet nevertheless the computational ability has already surpassed human being.